In [6]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString

In [35]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString

def convert_string(input_str):
    # Extract the date part using regular expression
    pattern = r'\d{2}/\d{2}/\d{4}'
    date_part = re.search(pattern, input_str).group()
    match = re.search(pattern, input_str)
    if match:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_part, '%d/%m/%Y')
        formatted_date = date_obj.strftime('%Y-%m-%d')
        return formatted_date
    else:
        return ""
# return the expected content in html format
def get_content_kenh14(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    article =  soup.find('div', class_ = "knc-content")
    h2_tag =  soup.find('h2', class_ = 'knc-sapo')
    article.insert(0, h2_tag)
    caption_text_list =article.find_all( class_="PhotoCMS_Caption")
    post_time = soup.find('span', class_ = 'kbwcm-time').text.strip()
    published_date = convert_string(post_time)
    title = soup.find('h1').text.strip()
    for caption in caption_text_list:
        img_container = caption.previous_sibling
        img = img_container.find('img')
        img['src'] = img['data-original']
        img
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        try:
            caption_text = NavigableString(caption.find('p').text.strip())
        except (IndexError, AttributeError):
            caption_text = ''
        caption_end = NavigableString("[/caption]")
        img.insert_before(caption_start)
        img.insert_after(caption_end)
        img.insert_after(caption_text)
        caption.decompose()
    for script_or_style in article(['script', 'style']):
            script_or_style.decompose() 
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            del i['id']
            del i['class']
            del i['style']
        except AttributeError:
            continue
        except TypeError:
            continue            
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
            # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip() 
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: kenh14.vn"  # Set the content of <i> tag
    # Append the <i> tag as the last child of the <article> tag
    article.append(source_tag)
    for i in article.find_all('div', {'align': 'center'}):
        i.decompose()
    for element in article.find_all(recursive = True,string=True):
        if isinstance(element, NavigableString) and element.strip() == '':
            element.extract()
    #remove element withou child element or have children element which is empty
    for i in article.find_all(recursive = True):
        if i.children == None and i.string == None:
            i.decompose()
    for i in article.find_all(recursive = True):
        try:
            i.text.trip()
        except AttributeError as e:
            continue
    return article, title, published_date 
def get_post(url):
    try:
        content,title,published_date = get_content_kenh14(url)
        return content,title,published_date
    except AttributeError as e:
        print(e)
def convert_time_string(posted_date):
    pattern = r'\d{2}/\d{2}/\d{4}'
    match = re.search(pattern, posted_date)

    if match:
        date_string = match.group()

        # Convert to datetime object
        datetime_obj = datetime.strptime(date_string, "%d/%m/%Y")
        return datetime_obj
    else:
        return ''
def get_list_url(cate_url):
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    list_url = soup.find_all('h3', class_ = "knswli-title")
    list = []
    for i in list_url:
        try:
            path = i.find('a')['href']
            url = 'https://kenh14.vn'+ path
            list.append(url)
        except TypeError:
            continue
    return list
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time() -3*24*3600)
    for i in urls:
        response = requests.get(i)
        soup = BeautifulSoup(response.content, 'html5lib')
        try:
            date_posted = soup.find('span', class_ = 'kbwcm-time').text.strip()
            date_posted_norm = convert_time_string(date_posted)
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(i)
                print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls
def add_list(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):  
            urls = get_list_url(web_json_obj['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_json_obj['urls'][i]['sub-category'][j]['url'])
            web_json_obj['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)
# add post content from get content function to json object
def add_post(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):
            web_json_obj['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_json_obj['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_json_obj['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_json_obj['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_json_obj['urls'][i]['sub-category'][j]['cate_id'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
#add all necessary information to json object
def get_news_kenh14():
    _kenh14 = {
            "home_page":"https://kenh14.vn/",
            "urls":{
                "sport":
                {
                 "url":"https://kenh14.vn/sport.chn",
                 "sub-category":{  
                    0:{"name":"Hậu trường",
                     "url":"https://kenh14.vn/sport/hau-truong.chn",
                     "cate_id":38,
                      "url_list" : []},
                    1:{"name":"Esports",
                     "url":"https://kenh14.vn/sport/esports.chn",
                     "cate_id":61,
                      "url_list" : []}
                 }
                }
            }
        }
#
    add_list(_kenh14)
    add_post(_kenh14)
    return _kenh14
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": 'draftpost',#'5goalvodichcmnl',  # Replace with your actual access token
        "published_date": published_date,
        "domain":"kenh14"
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    _kenh14 = get_news_kenh14()
    for i in list(_kenh14['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(_kenh14['urls'][i]['sub-category'].keys()):
            url_list =  _kenh14['urls'][i]['sub-category'][j]['url_list']
            for t in range(0,len(url_list)):
                content = _kenh14['urls'][i]['sub-category'][j]['content'][t]['text']
                title = _kenh14['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = _kenh14['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = _kenh14['urls'][i]['sub-category'][j]['cate_id']
                send_post_to_5goals(title,str(content), cate_id, published_date)
                time.sleep(5)
    


In [36]:
main()

sport 0 https://kenh14.vn/sport/hau-truong.chn
https://kenh14.vn/diem-chung-trong-cong-thuc-kiem-tien-cua-doan-hai-my-va-mai-ha-trang-lay-chong-cau-thu-noi-tieng-nhan-quang-cao-20231208160640468.chn
https://kenh14.vn/tranh-cai-chuyen-doan-van-hau-nhan-tin-lam-quen-doan-hai-my-khi-van-con-hen-ho-ban-gai-cu-20231208105938411.chn
sport 1 https://kenh14.vn/sport/esports.chn
https://kenh14.vn/khau-chien-voi-doi-thu-cuu-vo-dich-cktg-quyet-tuyet-tinh-luon-voi-lmht-bang-hanh-dong-tan-nhan-20231208111332979.chn
sport 0 38 Hậu trường Hậu trường Điểm chung trong công thức kiếm tiền của Doãn Hải My và Mai Hà Trang: Lấy chồng cầu thủ nổi tiếng, nhận quảng cáo https://kenh14.vn/diem-chung-trong-cong-thuc-kiem-tien-cua-doan-hai-my-va-mai-ha-trang-lay-chong-cau-thu-noi-tieng-nhan-quang-cao-20231208160640468.chn
sport 0 38 Hậu trường Hậu trường Tranh cãi chuyện Đoàn Văn Hậu nhắn tin làm quen Doãn Hải My khi vẫn còn hẹn hò bạn gái cũ https://kenh14.vn/tranh-cai-chuyen-doan-van-hau-nhan-tin-lam-quen-doan

In [26]:
_kenh14 = get_news_kenh14()

sport 0 https://kenh14.vn/sport/hau-truong.chn
https://kenh14.vn/thanh-chung-chup-trom-vo-mat-moc-mai-me-lam-cay-thong-noel-luc-nua-dem-nhan-sac-van-phai-cham-10-diem-20231209170547175.chn
https://kenh14.vn/me-vo-khang-dinh-doan-van-hau-may-man-moi-duoc-vao-nha-doan-hai-my-thai-do-cung-ran-voi-chang-re-qua-hai-chi-tiet-20231209084221904.chn
https://kenh14.vn/nguyen-van-toan-chu-tich-tuoi-25-thu-nhap-cao-nhat-nhi-lang-bong-da-nhung-van-tran-tro-vi-thieu-dieu-nay-20231209113138824.chn
https://kenh14.vn/doan-hai-my-tong-ket-tuan-trang-mat-tai-singapore-voi-doan-van-hau-chi-bang-mot-tam-anh-20231209084502849.chn
https://kenh14.vn/doan-van-hau-ke-ve-lan-dau-to-tinh-voi-hai-my-nhung-cang-noi-cang-sai-voi-ban-gai-cu-20231209074638014.chn
sport 1 https://kenh14.vn/sport/esports.chn
sport 0 38 Hậu trường Hậu trường Thành Chung chụp trộm vợ mặt mộc mải mê làm cây thông noel lúc nửa đêm, nhan sắc vẫn phải chấm 10 điểm https://kenh14.vn/thanh-chung-chup-trom-vo-mat-moc-mai-me-lam-cay-thong-noel-lu

In [27]:
_kenh14

{'home_page': 'https://kenh14.vn/',
 'urls': {'sport': {'url': 'https://kenh14.vn/sport.chn',
   'sub-category': {0: {'name': 'Hậu trường',
     'url': 'https://kenh14.vn/sport/hau-truong.chn',
     'cate_id': 38,
     'url_list': ['https://kenh14.vn/thanh-chung-chup-trom-vo-mat-moc-mai-me-lam-cay-thong-noel-luc-nua-dem-nhan-sac-van-phai-cham-10-diem-20231209170547175.chn',
      'https://kenh14.vn/me-vo-khang-dinh-doan-van-hau-may-man-moi-duoc-vao-nha-doan-hai-my-thai-do-cung-ran-voi-chang-re-qua-hai-chi-tiet-20231209084221904.chn',
      'https://kenh14.vn/nguyen-van-toan-chu-tich-tuoi-25-thu-nhap-cao-nhat-nhi-lang-bong-da-nhung-van-tran-tro-vi-thieu-dieu-nay-20231209113138824.chn',
      'https://kenh14.vn/doan-hai-my-tong-ket-tuan-trang-mat-tai-singapore-voi-doan-van-hau-chi-bang-mot-tam-anh-20231209084502849.chn',
      'https://kenh14.vn/doan-van-hau-ke-ve-lan-dau-to-tinh-voi-hai-my-nhung-cang-noi-cang-sai-voi-ban-gai-cu-20231209074638014.chn'],
     'content': {0: {'text': <div 

In [16]:
_kenh14 = get_news_kenh14()


sport 0 https://kenh14.vn/sport/hau-truong.chn
https://kenh14.vn/thanh-chung-chup-trom-vo-mat-moc-mai-me-lam-cay-thong-noel-luc-nua-dem-nhan-sac-van-phai-cham-10-diem-20231209170547175.chn
https://kenh14.vn/me-vo-khang-dinh-doan-van-hau-may-man-moi-duoc-vao-nha-doan-hai-my-thai-do-cung-ran-voi-chang-re-qua-hai-chi-tiet-20231209084221904.chn
https://kenh14.vn/nguyen-van-toan-chu-tich-tuoi-25-thu-nhap-cao-nhat-nhi-lang-bong-da-nhung-van-tran-tro-vi-thieu-dieu-nay-20231209113138824.chn
https://kenh14.vn/doan-hai-my-tong-ket-tuan-trang-mat-tai-singapore-voi-doan-van-hau-chi-bang-mot-tam-anh-20231209084502849.chn
https://kenh14.vn/doan-van-hau-ke-ve-lan-dau-to-tinh-voi-hai-my-nhung-cang-noi-cang-sai-voi-ban-gai-cu-20231209074638014.chn
sport 1 https://kenh14.vn/sport/esports.chn
sport


In [32]:
for i in list(_kenh14['urls'].keys()):
#web_24h_com_vn2['url'][i]['cate_id']
    for j in list(_kenh14['urls'][i]['sub-category'].keys()):
        #print(_kenh14['urls'][i]['sub-category'][j]['url_list'])
        url_list =  _kenh14['urls'][i]['sub-category'][j]['url_list']
        for t in range(0,len(url_list)):
            content = _kenh14['urls'][i]['sub-category'][j]['content'][t]['text']
            print(content)

<div class="knc-content"><h2>
                                    Thành Chung chia sẻ ngủ một giấc dậy vẫn thấy vợ loay hoay làm cây thông Noel.
                                </h2><!-- Kham pha --><p>Giáng sinh 2023 sắp đến, hòa chung trong không khí tưng bừng đón giáng sinh, vợ của hậu vệ Nguyễn Thành Chung cũng mua sắm rất nhiều đồ để trang hoàng không gian trong nhà. Ngoài cây thông noel lớn đã hoàn thành chơi được vài ngày, Tố Uyên lại mới tự tay làm thêm một cây thông nhỏ.</p><p>Hình ảnh được hậu vệ Thành Chung chụp lại lúc nửa đêm, chàng cầu thủ chia sẻ:</p><p>"Vâng tôi ngủ được một giấc dậy vẫn thấy cô ấy đang loay hoay với tác phẩm của mình".</p><p>Có thể thấy Tố Uyên là một cô gái rất thích trang trí nhà cửa, làm đẹp cho tổ ấm của mình. Đặc biệt điều khiến dân tình chú ý nhất là nhan sắc của nàng WAG sinh năm 2000. Dù bị chồng chụp lén xuất hiện với gương mặt ngơ ngác nhưng nhan sắc của cô gái đến từ Tuyên Quang vẫn "không thể đùa được". Ngô Tố Uyên sở hữu gương mặt baby, là

In [34]:
main()

sport 0 https://kenh14.vn/sport/hau-truong.chn
https://kenh14.vn/diem-chung-trong-cong-thuc-kiem-tien-cua-doan-hai-my-va-mai-ha-trang-lay-chong-cau-thu-noi-tieng-nhan-quang-cao-20231208160640468.chn
https://kenh14.vn/tranh-cai-chuyen-doan-van-hau-nhan-tin-lam-quen-doan-hai-my-khi-van-con-hen-ho-ban-gai-cu-20231208105938411.chn
sport 1 https://kenh14.vn/sport/esports.chn
https://kenh14.vn/khau-chien-voi-doi-thu-cuu-vo-dich-cktg-quyet-tuyet-tinh-luon-voi-lmht-bang-hanh-dong-tan-nhan-20231208111332979.chn
sport 0 38 Hậu trường Hậu trường Điểm chung trong công thức kiếm tiền của Doãn Hải My và Mai Hà Trang: Lấy chồng cầu thủ nổi tiếng, nhận quảng cáo https://kenh14.vn/diem-chung-trong-cong-thuc-kiem-tien-cua-doan-hai-my-va-mai-ha-trang-lay-chong-cau-thu-noi-tieng-nhan-quang-cao-20231208160640468.chn
sport 0 38 Hậu trường Hậu trường Tranh cãi chuyện Đoàn Văn Hậu nhắn tin làm quen Doãn Hải My khi vẫn còn hẹn hò bạn gái cũ https://kenh14.vn/tranh-cai-chuyen-doan-van-hau-nhan-tin-lam-quen-doan

AttributeError: 'dict' object has no attribute 'key'

In [7]:
get_content_kenh14('https://kenh14.vn/dt-nhat-ban-bi-chi-trich-vi-chon-doi-hinh-b-choi-tran-khai-xuan-voi-dt-thai-lan-20231208093013832.chn')[0]

<div class="knc-content"><h2>
                                    Nobuhiro Takeda, cựu tuyển thủ Nhật Bản và hiện là một bình luận viên bóng đá, đã bày tỏ sự không hài lòng khi ĐT Nhật Bản chọn Thái Lan để chơi trận giao hữu đầu năm 2024.
                                </h2><!-- Kham pha --><div type="Photo"><div>[caption id="" align="aligncenter" width="800"]<img alt="ĐT Nhật Bản bị chỉ trích vì chọn đội hình B chơi trận khai xuân với ĐT Thái Lan - Ảnh 1." class="aligncenter" data-original="https://kenh14cdn.com/203336854389633024/2023/12/8/photo-2-17020025466941548246883.jpg" h="481" height="400" photoid="657409107047018496" rel="lightbox" src="https://kenh14cdn.com/203336854389633024/2023/12/8/photo-2-17020025466941548246883.jpg" title="ĐT Nhật Bản bị chỉ trích vì chọn đội hình B chơi trận khai xuân với ĐT Thái Lan - Ảnh 1." type="photo" w="645" width="800"/>[/caption]</div></div><p>Vào ngày 1/1/2024, ĐT Nhật Bản sẽ đọ sức Thái Lan. Đây là màn tập dượt để họ chuẩn bị cho Asian Cup 

In [47]:
response = requests.get('https://kenh14.vn/me-vo-tre-dep-cua-doan-van-hau-muon-dap-di-xay-lai-ngoai-hinh-ket-qua-ra-sao-20231207192943245.chn')
soup = BeautifulSoup(response.content, 'html.parser')
article =  soup.find('div', class_ = "knc-content")
h2_tag =  soup.find('h2', class_ = 'knc-sapo')
article.insert(0, h2_tag)
caption_text_list =article.find_all( class_="PhotoCMS_Caption")
for caption in caption_text_list:
    img_container = caption.previous_sibling
    img = img_container.find('img')
    img['src'] = img['data-original']
    caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
    try:
        caption_text = NavigableString(caption.find('p').text.strip())
    except IndexError:
        caption_text = ''
    caption_end = NavigableString("[/caption]")
    img.insert_before(caption_start)
    img.insert_after(caption_end)
    img.insert_after(caption_text)
    caption.decompose()
"""children = article.find_all(recursive=False)
related_news =  article.find('div', class_ = 'knc-relate-wrapper relationnews')
related_news.decompose()
for i in children[-6:]:
    i.decompose() 
img_list = article.find_all(['img','video'])[:-1]
n_img = len(img_list)
for i in range(0,n_img):
    caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
    caption_text = NavigableString(caption_text_list[i].text)
    caption_end = NavigableString("[/caption]")
    caption_text_list[i].decompose()
        # Insert the custom tags and caption text around the <img> tag
    img_list[i].insert_before(caption_start)
    img_list[i].insert_after(caption_end)
    try:
        img_list[i].insert_after(caption_text) 
    except IndexError as e:
        print(e)        #print(len(caption_text_list))"""

'children = article.find_all(recursive=False)\nrelated_news =  article.find(\'div\', class_ = \'knc-relate-wrapper relationnews\')\nrelated_news.decompose()\nfor i in children[-6:]:\n    i.decompose() \nimg_list = article.find_all([\'img\',\'video\'])[:-1]\nn_img = len(img_list)\nfor i in range(0,n_img):\n    caption_start = NavigableString("[caption id="" align="aligncenter" width="800"]")\n    caption_text = NavigableString(caption_text_list[i].text)\n    caption_end = NavigableString("[/caption]")\n    caption_text_list[i].decompose()\n        # Insert the custom tags and caption text around the <img> tag\n    img_list[i].insert_before(caption_start)\n    img_list[i].insert_after(caption_end)\n    try:\n        img_list[i].insert_after(caption_text) \n    except IndexError as e:\n        print(e)        #print(len(caption_text_list))'

In [38]:
article

<div class="knc-content"><h2 class="knc-sapo">
                                    Tuyên bố muốn phẫu thuật thẩm mỹ nhưng bà Đoàn Phương Mai - mẹ vợ của hậu vệ Đoàn Văn Hậu - vẫn chưa đạt được như ý nguyện?
                                </h2>
<!-- Kham pha -->
<p>Bà Đoàn Phương Mai (nickname Mai Đoàn) - mẹ của nàng WAG Doãn Hải My - thời gian gần đây nổi rần rần trên mạng xã hội khi xuất hiện với diện mạo trẻ đẹp trong đám cưới của con gái và hậu vệ ĐT Việt Nam Đoàn Văn Hậu. Ở độ tuổi 46, bà Mai Đoàn vẫn thu hút sự chú ý với nhan sắc mặn mà, vóc dáng nuột nà cùng phong cách thời trang sành điệu. Khi bà Mai Đoàn đi cạnh còn gái Doãn Hải My, hai người thường được nhận xét giống chị em hơn là mẹ con.</p><p>Dù được khen về ngoại hình nhưng mẹ vợ của Đoàn Văn Hậu vẫn có điểm chưa hài lòng. Một thời gian trước, trên Facebook cá nhân, bà Mai Đoàn đã chia sẻ chuyện muốn tác động dao kéo để trùng tu nhan sắc. Mẹ Doãn Hải My viết: "<em>Muốn đập đi làm lại bao thứ: độ loa này, hút mỡ này, siết 

In [ ]:
def get_list_url(cate_url):
    response = requests.get(cate_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    list_url = soup.find_all('h3', class_ = "knswli-title")
    list = []
    for i in list_url:
        try:
            path = i.find('a')['href']
            url = 'https://kenh14.vn'+ path
            list.append(url)
        except TypeError:
            continue
    return list